# 5장

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 5.1 계산 그래프

- 계산 그래프 = 계산 과정을 그래프로 나타낸 것.
- 노드와 엣지로 표현현

### 5.1.1 계산 그래프로 풀다

- 계산 과정을 노드와 화살표로 표현.
- 계산 결과를 화살표 위에 적어 각 노드의 계산 결과가 왼쪽에서 오른쪽으로 전해짐.

- '**순전파**' : 계산을 왼쪽에서 오른쪽으로 진행.
- '**역전파**' : 계산을 오른쪽에서 왼쪽으로 진행.

### 5.1.2 국소적 계산

- '국소적 계산'을 전파함으로써 최종 결과를 얻는다
- 국소적? -> 자신과 직접 관계된 작은 범위

### 5.1.3 왜 계산 그래프로 푸는가?

- 역전파를 통해 '미분'을 효율적으로 계산할 수 있음.

## 5.2 연쇄법칙

- 역전파는 '국소적인 미분'을 순방향과는 반대인 오른쪽에서 왼쪽으로 전달.
- **연쇄법칙** : '국소적인 미분'을 전달하는 원리

### 5.2.1 계산 그래프의 역전파

- 역전파의 계산 절차 : 신호 E에 노드의 국소적 미분을 곱한 후 다음 노드로 전달하는 것

### 5.2.2 연쇄법칙이란?

- 합성 함수의 미분에 대한 성질
- 합성 함수의 미분 : 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

### 5.2.3 연쇄법칙과 계산 그래프

[노션 참고]

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

상류층의 신호에 1을 곱하여 보냄. 즉, 받은 신호 그대로 흘려보냄.

### 5.3.2 곱셈 노드의 역전파

입력 신호들을 '서로 바꾼 값'을 곱해서 하류로 보냄.

### 5.3.3 사과 쇼핑의 예

[노션 참고]

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [2]:
class MulLayer:
    def __init__(self): # x와 y 초기화 -> 순전파 시의 입력 값을 유지하기 위해 사용
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x* y # 곱해서 반환
    
        return out
    
    def backward(self, dout):
        dx = dout * self.y # x와 y를 바꾼다.
        dy = dout * self.x
        
        return dx, dy

사과 쇼핑 구현

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [4]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 5.4.2 덧셈 계층

In [5]:
class AddLayer:
    def __init__(self):
        pass # 아무것도 하지 않음
    
    def forward(self, x, y):
        out = x + y # 더해서 흘림.
        return out
    
    def backward(self, dout):
        dx = dout * 1 # 그냥 흘림.
        dy = dout * 1
        return dx, dy

사과 2개와 귤 3개 구입 구현

In [6]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num) # (1)
orange_price = mul_orange_layer.forward(orange, orange_num) # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price) # (3)
price = mul_tax_layer.forward(all_price, tax) # (4)

# 역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice) # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price) # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price) # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price) # (1)

print(price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

In [7]:
class Relu:
    def __init__(self):
        self.mask = None
        # mask : True/False로 구성된 넘파이 배열
        
    def forward(self, x):
        self.mask = (x <= 0) 
        out = x.copy() # 0보다 크면 그대로 출력
        out[self.mask] = 0 # 0보다 작으면 0으로 출력
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

mask 변수 설명

In [8]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [9]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


### 5.5.2 Sigmoid 계층

In [10]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1+np.exp(-x))
        self.out = out
        
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층

In [11]:
X = np.random.rand(2) # 입력
W = np.random.rand(2, 3) # 가중치
B = np.random.rand(3) # 편향

print(X.shape) # (2,)
print(W.shape) # (2, 3)
print(B.shape) # (3,)

Y = np.dot(X, W) + B
print('\n',Y)

(2,)
(2, 3)
(3,)

 [0.15971884 0.40943738 0.60900002]


### 5.6.2 배치용 Affine 계층

**순전파**

In [12]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [13]:
X_dot_W + B # 순전파 때는 각각의 데이터에 편향이 더해짐.

array([[ 1,  2,  3],
       [11, 12, 13]])

**역전파**

In [14]:
dY = np.array([[1, 2, 3], [4, 5, 6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [15]:
dB = np.sum(dY, axis = 0) # 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더함.
dB

array([5, 7, 9])

**구현**

In [16]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx

### 5.6.3 Softmax-with-Loss 계층

In [17]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

## 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림

오차역전파법이 등장하는 단계는 '기울기 산출'이다.

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

2층 신경망을 TwoLayerNet 클래스로 구현

In [18]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b2'] = np.zeros(output_size)
        
        
        # 계층 생성
        self.layers = OrderedDict() # 순서가 있는 딕셔너리
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x # 순서대로 forward 호출
    
    # x : 입력 데이터, t : 정답레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

- 수치 미분은 느리지만, 오차역전파법을 정확히 구현했는지 확인하기 위해 필요.
- **기울기 확인** : 수치 미분의 결과와 오차역전파법의 결과를 비교.

In [21]:
from datasets.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key])) # 가중치 - 편향
    print(key + ":" + str(diff))
    


W1:2.117877832098349e-10
b1:1.1824881272347754e-09
W2:6.739515539258557e-08
b2:1.3581318691613386e-07


기울기 차이가 매우 작다.

### 5.7.4 오차역전파법을 사용한 학습 구현하기 

In [23]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from datasets.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.10523333333333333 0.1087
0.7997333333333333 0.8012
0.8791 0.8818
0.899 0.902
0.9076166666666666 0.9111
0.9148 0.9171
0.9198833333333334 0.923
0.92365 0.925
0.9272666666666667 0.9285
0.93095 0.9317
0.9340666666666667 0.9355
0.9362666666666667 0.9361
0.9394166666666667 0.9379
0.94115 0.9399
0.9435 0.9416
0.9449833333333333 0.9429
0.94685 0.9453


## 5.8 정리

**이번 장에서 배운 내용**
- 계산 그래프를 이용하면 계산 과정을 시각적으로 파악할 수 있다.
- 계산 그래프의 노드는 국소적 계산으로 구성된다. 국소적 계산을 조합해 전체 계산을 구성한다.
- 계산 그래프의 순전파는 통상의 계산을 수행한다. 한편, 계산 그래프의 역전파로는 각 노드의 미분을 구할 수 있다.
- 신경망의 구성 요소를 계층으로 구현하여 기울기를 효율적으로 계산할 수 있다.(오차역전파법)
- 수치 미분과 오차역전파법의 결과를 비교하면 오차역전파법의 구현에 잘못이 없는지 확인할 수 있다.(기울기 확인)